# Function Calling 사용자 정의 함수 사용


# 🤖 OpenAI Function Calling & Assistant API

## 🧠 전체 개요: 이건 무엇을 하는 코드인가요?

이 코드는 "GPT와 대화를 하면서, 내가 만든 **함수를 자동으로 호출**해서 결과를 받아주는 시스템"을 구현한 것입니다.

마치 스마트한 비서가 나에게 맞춤형 대답을 주기 위해 상황에 따라 미리 준비된 계산기, 지도 앱, 날씨 앱 등을 자동으로 꺼내 쓰는 것과 비슷합니다!

---

## 🧩 1. 전체 구조 요약

> 🔧 **핵심 아이디어**:  
> GPT에게 "함수를 소개"해주면, 사용자의 질문에 따라 **적절한 함수**를 **자동으로 골라 호출하고**, 그 결과를 바탕으로 대답을 합니다.


💬 사용자 질문
→ 🧠 GPT가 "어떤 함수가 필요할까?" 판단
→ 🔍 필요한 인자 추출 (예: 서울, 섭씨)
→ 🔁 실제 Python 함수 호출
→ ✏️ 결과를 GPT가 다시 읽고 자연어로 정리해 답변


---

## 🔂 2. 코드의 흐름 (기본 Function Calling 방식 기준)

1. **함수를 정의**합니다. (예: `get_current_weather`)
2. GPT에게 **사용자 질문과 함께** 함수 목록을 전달합니다.
3. GPT가 필요한 함수를 선택하고, 필요한 인자를 뽑습니다.
4. GPT가 선택한 함수를 **Python 코드로 직접 실행**합니다.
5. 실행 결과를 GPT에게 넘깁니다.
6. GPT가 결과를 읽고 최종적으로 **자연어로 설명**합니다.

---

## 🛠 3. 다양한 예제 유형들

### 🌤 날씨 API (Function Calling 기본 예시)
- 질문: “서울의 오늘 날씨는?”
- GPT가 `get_current_weather(location="서울")` 함수 호출
- 결과: “서울은 28도이며, 맑고 습합니다.”

### 🍽 음식점 추천 / 📚 책 추천 / 🎬 영화 추천
- GPT가 적절한 추천 함수를 골라서 호출함

### 🌐 Wikipedia 검색 예제
- 질문: “광안대교 개통일은?”
- GPT가 `search_wikipedia(query="광안대교", lang="ko")` 호출
- Wikipedia에서 찾아와서 요약해 줌

---


Function Calling은 사용자가 미리 function(기능)들을 정의해놓고 질문이 들어왔을때 LLM이 적합한 Function을 선택해 실행하도록 하는 기능임


Function Calling 과정


질문 → LLM이 함수 선택 → 인자 추출 후 함수 호출 → 답변




1,질문: 사용자가 질문을 입력.

2.LLM이 적합한 함수 선택: LLM이 질문을 분석하고, 적절한 함수를 찾아냄.

3.인자 추출 후 함수 호출: 질문에서 필요한 인자를 추출해 함수를 호출.

4/답변 제공: 함수 결과를 바탕으로 답변 생성.

Function Calling 1

In [ ]:
!pip install openai==0.28

In [ ]:
import openai  # OpenAI API를 사용하기 위한 라이브러리
import os      # 운영 체제와 관련된 설정을 하기 위한 표준 모듈

# ✅ 포인트: API 키는 GPT 모델과 통신할 수 있도록 하는 인증 수단입니다.
# API 키를 환경 변수에 저장해 보안적으로 조금 더 안전하게 관리합니다.
# os.environ['OPENAI_API_KEY'] = "sk-..."  # OpenAI에서 발급받은 개인 API 키 설정 (공개 금지)
# openai.api_key = os.getenv('OPENAI_API_KEY')  # 환경 변수에서 API 키를 불러와 설정합니다.

# ✅ 포인트: GPT에게 처음으로 질문을 보내는 기본적인 예시입니다.
# 💡 예시: 사용자가 "서울 날씨 알려줘"라고 말하는 것과 같습니다.
completion = openai.ChatCompletion.create(
    model="gpt-4o",  # 사용할 모델(gpt-4o)은 최신 GPT-4 모델 중 하나입니다.
    messages=[{"role": "user", "content": "오늘 서울의 실시간 날씨 정보는?"}],  # 사용자 메시지를 GPT에 전달합니다.
)

# GPT가 생성한 응답을 가져옵니다.
reply_content = completion.choices[0].message.content  # 응답 메시지 내용 추출

# 결과 출력: GPT가 생성한 자연어 응답을 출력합니다.
print(reply_content)  # 예: "오늘 서울은 맑고 기온은 28도입니다."

죄송합니다만, 저는 실시간 데이터를 제공할 수 없습니다. 서울의 실시간 날씨를 확인하려면 기상청 웹사이트나 모바일 앱, 혹은 다른 신뢰할 수 있는 날씨 서비스(예: 웨더닷컴, 아큐웨더 등)에 접속하는 것이 좋습니다.


In [ ]:
import openai
import json  # 함수 결과를 JSON 형식으로 만들기 위해 필요한 모듈

# ✅ 포인트: 실제 날씨 데이터를 대신할 테스트용 함수입니다.
# 사용자가 요청한 지역의 날씨 정보를 미리 정의된 값으로 반환합니다.
def get_current_weather(location, unit="섭씨"):
    # 💡 예시: location="서울"이라면 아래와 같은 날씨 정보가 반환됩니다.
    return json.dumps({
        "location": location,          # 사용자가 요청한 지역명 (예: 서울)
        "temperature": "28",           # 현재 온도 (예시 값)
        "unit": unit,                  # 온도 단위: 섭씨(default) 또는 화씨
        "forecast": ["sunny", "wet"],  # 간단한 예보 (맑음 + 약간 습함)
    })

In [ ]:
import json


In [ ]:
# 사용자로부터 받은 메시지를 대화 형식으로 저장합니다.
messages = [{"role": "user", "content": "오늘 서울의 실시간 날씨 정보는?"}]

# ✅ 포인트: GPT에게 어떤 함수를 사용할 수 있는지 소개해주는 역할
# GPT는 이 정보를 바탕으로 "이 질문에는 어떤 함수가 적절할까?"를 판단합니다.
functions = [
    {
        "name": "get_current_weather",  # 호출 가능한 함수 이름
        "description": "특정 지역의 현재 날씨를 알려줍니다.",  # 이 함수가 하는 일에 대한 설명
        "parameters": {  # 함수에 전달해야 하는 인자(입력값) 정의
            "type": "object",  # JSON 객체 형태로 전달
            "properties": {
                "location": {"type": "string", "description": "지역 이름, 예: 서울"},  # 필수 입력값
                "unit": {"type": "string", "enum": ["섭씨", "화씨"]},  # 선택값 (섭씨 or 화씨)
            },
            "required": ["location"],  # 반드시 필요한 값은 'location'
        },
    }
]

In [ ]:
# ✅ 포인트: 이제 GPT에게 메시지와 함께 사용할 수 있는 함수 정보도 같이 보냅니다.
# function_call="auto"는 GPT가 필요하면 직접 함수 호출을 시도할 수 있게 해줍니다.
response = openai.ChatCompletion.create(
    model="gpt-4o",        # 사용할 GPT 모델
    messages=messages,     # 대화 내용
    functions=functions,   # 함수 정의 목록
    function_call="auto",  # 함수 자동 호출 허용
)


In [ ]:
# GPT의 응답 전체를 출력 (디버깅 또는 로직 확인용)
print(response)

{
  "id": "chatcmpl-AAY77xcmr0AcxX2ib5Sx7n4LLYVty",
  "object": "chat.completion",
  "created": 1727077777,
  "model": "gpt-4o-2024-05-13",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\"location\":\"\uc11c\uc6b8\",\"unit\":\"\uc12d\uc528\"}"
        },
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 79,
    "completion_tokens": 20,
    "total_tokens": 99,
    "completion_tokens_details": {
      "reasoning_tokens": 0
    }
  },
  "system_fingerprint": "fp_e375328146"
}


In [ ]:

# ✅ 포인트: GPT가 실제로 어떤 함수를 호출했는지 확인
response_message = response["choices"][0]["message"]
print(response_message)


{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\":\"\uc11c\uc6b8\",\"unit\":\"\uc12d\uc528\"}"
  },
  "refusal": null
}


In [ ]:
# ✅ 포인트: GPT가 '함수 호출'을 시도했는지 확인하는 조건문
# function_call 필드가 있으면, GPT가 자동으로 함수 호출을 요청한 것임
if response_message.get("function_call"):
    function_name = response_message["function_call"]["name"]  # 요청한 함수 이름 추출
    function_args = json.loads(response_message["function_call"]["arguments"])  # 함수에 넘길 인자 추출

    # ✅ 포인트: GPT가 요청한 대로 실제 파이썬 함수 실행 (함수 연결 단계)
    function_response = get_current_weather(**function_args)  # 예: get_current_weather(location="서울")

    # ✅ 포인트: GPT가 요청했던 메시지를 대화 기록에 추가 (추후 맥락 유지)
    messages.append(response_message)  # GPT의 함수 호출 메시지 추가
    messages.append({
        "role": "function",       # 이건 함수 결과임을 표시
        "name": function_name,    # 어떤 함수의 결과인지 명시
        "content": function_response,  # 함수 실행 결과(JSON 포맷의 날씨 정보)
    })

    # ✅ 포인트: 함수 실행 결과를 포함한 전체 대화를 다시 GPT에게 보내서, 최종 응답 생성
    second_response = openai.ChatCompletion.create(
        model="gpt-4o",  # 동일한 모델 사용
        messages=messages,  # 전체 대화 기록 전달
    )

    # ✅ 포인트: 최종적으로 GPT가 자연어로 정리해서 사용자에게 보여줄 응답
    print(second_response["choices"][0]["message"]["content"])  # 예: "서울은 28도이며 맑습니다."

현재 서울의 기온은 섭씨 28도이며, 날씨는 맑고 습한 상태입니다. 추가적인 정보가 필요하시면 알려주세요!


Function Calling(Def 함수 정의 없이 Functions만 정의만 경우)

Function Calling 2

In [ ]:
import openai  # OpenAI API를 사용하기 위한 모듈
import json  # JSON 데이터를 다루기 위한 모듈

# 1. 음식점 추천 정보를 반환하는 함수 정의
def get_restaurant_recommendation(location, cuisine="한식"):
    # 특정 위치와 음식 종류에 따른 음식점 추천 정보를 JSON 형식으로 반환하는 함수입니다.
    return json.dumps({
        "location": location,  # 지역 이름
        "cuisine": cuisine,  # 음식 종류 (기본값은 한식)
        "recommendations": [  # 추천 음식점 목록
            {"name": "김치찜 전문점", "rating": 4.5, "address": "서울 강남구"},  # 첫 번째 음식점 정보
            {"name": "비빔밥 집", "rating": 4.2, "address": "서울 마포구"},  # 두 번째 음식점 정보
        ],
    })

# 2. 도서 추천 정보를 반환하는 함수 정의
def get_book_recommendation(genre="소설"):
    # 특정 장르에 따른 도서 추천 정보를 JSON 형식으로 반환하는 함수입니다.
    return json.dumps({
        "genre": genre,  # 도서 장르 (기본값은 소설)
        "recommendations": [  # 추천 도서 목록
            {"title": "그릿", "author": "앤젤라 더크워스", "rating": 4.7},  # 첫 번째 도서 정보
            {"title": "나미야 잡화점의 기적", "author": "히가시노 게이고", "rating": 4.9},  # 두 번째 도서 정보
        ],
    })

# 3. 영화 추천 정보를 반환하는 함수 정의
def get_movie_recommendation(genre="액션"):
    # 특정 장르에 따른 영화 추천 정보를 JSON 형식으로 반환하는 함수입니다.
    return json.dumps({
        "genre": genre,  # 영화 장르 (기본값은 액션)
        "recommendations": [  # 추천 영화 목록
            {"title": "인셉션", "director": "크리스토퍼 놀란", "rating": 4.8},  # 첫 번째 영화 정보
            {"title": "어벤져스: 엔드게임", "director": "안소니 루소, 조 루소", "rating": 4.7},  # 두 번째 영화 정보
        ],
    })


In [ ]:
functions = [
    {
        "name": "get_restaurant_recommendation",  # 함수 이름 설정
        "description": "특정 위치에서 사용자의 취향에 맞는 식당을 추천합니다.",  # 함수의 역할 설명
        "parameters": {  # 함수가 필요로 하는 인자 설정
            "type": "object",  # 인자의 데이터 형식을 객체(딕셔너리)로 설정
            "properties": {  # 인자의 세부 항목 정의
                "location": {"type": "string", "description": "지역 이름, 예: 서울"},  # 지역 이름 인자 설정
                "cuisine": {"type": "string", "description": "음식 종류, 예: 한식, 일식, 중식"},  # 음식 종류 인자 설정
            },
            "required": ["location"],  # 필수적으로 필요한 인자는 'location'
        },
    },
    {
        "name": "get_book_recommendation",  # 함수 이름 설정
        "description": "사용자의 선호 장르에 따라 도서를 추천합니다.",  # 함수의 역할 설명
        "parameters": {  # 함수가 필요로 하는 인자 설정
            "type": "object",  # 인자의 데이터 형식을 객체(딕셔너리)로 설정
            "properties": {  # 인자의 세부 항목 정의
                "genre": {"type": "string", "description": "도서 장르, 예: 소설, 자기계발, 역사"},  # 도서 장르 인자 설정
            },
            "required": ["genre"],  # 필수적으로 필요한 인자는 'genre'
        },
    },
    {
        "name": "get_movie_recommendation",  # 함수 이름 설정
        "description": "사용자의 선호 장르에 따라 영화를 추천합니다.",  # 함수의 역할 설명
        "parameters": {  # 함수가 필요로 하는 인자 설정
            "type": "object",  # 인자의 데이터 형식을 객체(딕셔너리)로 설정
            "properties": {  # 인자의 세부 항목 정의
                "genre": {"type": "string", "description": "영화 장르, 예: 액션, 드라마, 코미디"},  # 영화 장르 인자 설정
            },
            "required": ["genre"],  # 필수적으로 필요한 인자는 'genre'
        },
    }
]


In [ ]:
# 2. 사용자 요청과 함수 정보 설정
messages = [{"role": "user", "content": "액션 영화를 추천해줘!"}]

In [ ]:
# 3. GPT에 사용자 요청과 함수 정보를 보내기
response = openai.ChatCompletion.create(
    model="gpt-4o",
    messages=messages,
    functions=functions,
    function_call="auto",
)


In [ ]:
# 4. GPT가 요청한 함수 실행
response_message = response["choices"][0]["message"]
# GPT의 응답 중 첫 번째 선택지에서 메시지를 가져옵니다. 이 메시지는 GPT가 함수 호출을 요청했는지 여부를 확인하는 데 사용됩니다.

# GPT가 함수 호출을 요청했는지 확인
if response_message.get("function_call"):
    # GPT가 호출하려는 함수의 이름과 인자(필요한 정보)를 가져옵니다.
    function_name = response_message["function_call"]["name"]
    print("선택함수:", function_name)  # 호출된 함수 이름을 출력하여 확인합니다.
    function_args = json.loads(response_message["function_call"]["arguments"])
    # GPT가 요청한 함수 호출에 필요한 인자를 JSON 형식에서 파이썬 딕셔너리로 변환합니다.
    # 요청된 함수 이름에 따라 실제로 함수를 실행합니다
    if function_name == "get_restaurant_recommendation":
        # 만약 GPT가 'get_restaurant_recommendation' 함수를 호출했다면, 해당 함수를 실행합니다.
        function_response = get_restaurant_recommendation(**function_args)
    elif function_name == "get_book_recommendation":
        # 만약 GPT가 'get_book_recommendation' 함수를 호출했다면, 해당 함수를 실행합니다.
        function_response = get_book_recommendation(**function_args)
    elif function_name == "get_movie_recommendation":
        # 만약 GPT가 'get_movie_recommendation' 함수를 호출했다면, 해당 함수를 실행합니다.
        function_response = get_movie_recommendation(**function_args)

    # 5. 함수 실행 결과를 다시 GPT에게 전달
    messages.append(response_message)  # GPT의 요청 메시지를 대화 기록에 추가합니다.
    messages.append({
        "role": "function",  # 이 메시지의 역할(role)을 "function"으로 설정합니다.
        "name": function_name,  # 실행된 함수의 이름을 기록합니다.
        "content": function_response,  # 함수 실행 결과를 기록합니다.
    })

    # GPT에게 함수 실행 결과를 바탕으로 추가 응답을 요청합니다
    second_response = openai.ChatCompletion.create(
        model="gpt-4o",  # GPT 모델을 지정합니다.
        messages=messages,  # 지금까지의 대화 기록을 GPT에게 보냅니다.
    )

    # 최종적으로 GPT의 응답을 출력합니다 (예: 추천 결과를 보여줌)
    print(second_response["choices"][0]["message"]["content"])
    # GPT가 생성한 최종 응답을 출력합니다.


선택함수: get_movie_recommendation
액션 영화를 추천해드릴게요!

1. **인셉션**
   - 감독: 크리스토퍼 놀란
   - 평점: 4.8

2. **어벤져스: 엔드게임**
   - 감독: 안소니 루소, 조 루소
   - 평점: 4.7

이 두 영화는 액션과 흥미진진한 스토리가 잘 어우러져 있어 꼭 한 번 보시길 추천드립니다!


In [ ]:
import openai
import json

# 6. 사용자 메시지 설정: 사용자가 GPT에게 질문
messages = [{"role": "user", "content": "오늘 서울의 실시간 날씨 정보는?"}]

# 7. 함수 정보 설정: GPT에게 호출할 함수 정보 전달
functions = [
    {
        "name": "get_current_weather",  # 함수 이름
        "description": "특정 지역의 현재 날씨를 알려줍니다.",  # 함수 설명
        "parameters": {
            "type": "object",  # 함수가 받는 파라미터 형태 (JSON 형식)
            "properties": {
                "location": {"type": "string", "description": "지역 이름, 예: 서울"},  # 지역 정보
                "unit": {"type": "string", "enum": ["섭씨", "화씨"]},  # 온도 단위 (섭씨 또는 화씨)
            },
            "required": ["location"],  # 필수 파라미터는 'location'
        },
    }
]

# 8. GPT에게 메시지와 함수 정보를 전달하여 함수 호출 요청
response = openai.ChatCompletion.create(
    model="gpt-4",  # 사용할 GPT 모델
    messages=messages,  # 사용자의 질문 메시지
    functions=functions,  # GPT가 사용할 함수 정보
    function_call="auto",  # 필요 시 GPT가 함수를 자동으로 호출하게 설정
)

# 9. GPT의 응답에서 함수 호출 여부 확인
response_message = response["choices"][0]["message"]

# GPT가 함수 호출을 요청했는지 확인
if response_message.get("function_call"):
    function_name = response_message["function_call"]["name"]  # 호출하려는 함수 이름 추출
    function_args = json.loads(response_message["function_call"]["arguments"])  # 함수 호출에 필요한 인자 추출

    # 예시로 함수 실행 결과를 반환
    function_response = json.dumps({
        "location": function_args["location"],
        "temperature": "28",
        "unit": function_args.get("unit", "섭씨"),
        "forecast": ["sunny", "wet"],
    })

    # 10. 함수 실행 결과를 대화 기록에 추가
    messages.append(response_message)  # GPT의 함수 요청 메시지를 대화 기록에 추가
    messages.append({
        "role": "function",  # 이 메시지는 함수 실행 결과를 나타냄
        "name": function_name,  # 실행된 함수 이름
        "content": function_response,  # 함수 실행 결과 (날씨 정보)
    })

    # 11. 대화 기록을 기반으로 GPT에게 추가 응답 요청
    second_response = openai.ChatCompletion.create(
        model="gpt-4",  # 동일한 GPT 모델
        messages=messages,  # 전체 대화 기록 전달
    )

    # 12. 최종 응답 출력
    print(second_response["choices"][0]["message"]["content"])  # 최종적으로 GPT의 응답을 출력


오늘 서울의 현재 기온은 28도로, 날씨는 맑고 습도가 높습니다.


# FunctionCall(Wikipedia)

In [ ]:
!pip install wikipedia

In [ ]:
import os
import openai
import os
import requests
import json
import openai
# # API 키 설정
# os.environ['OPENAI_API_KEY'] = "A"  # OpenAI API 키를 환경 변수에 설정

# openai.api_key = os.getenv('OPENAI_API_KEY')


In [ ]:
import wikipedia
wikipedia.set_lang("ko") #en
r = wikipedia.search("축구")
print(r)#검색 결과는 검색 관련 내용이 있는 Wikipedia 페이지 제목 10개

['축구', '국제 축구 연맹', '대한민국 축구 국가대표팀', '대한축구협회', '유럽 축구 연맹', '아시아 축구 연맹', '올림픽 축구', '북중미카리브 축구 연맹', '남미 축구 연맹', '아프리카 축구 연맹']


In [ ]:
# 검색 내용과 언어를 입력받으면 Wikipedia를 검색해서 가장 먼저 나오는 페이지의 내용을 반환하는 함수
def search_wikipedia(query, lang):
    wikipedia.set_lang(lang)  # Wikipedia의 언어 설정, 사용자가 입력한 언어 코드로 설정합니다.
    r = wikipedia.search(query, results=1)  # 사용자가 입력한 검색어(query)를 Wikipedia에서 검색하고, 결과 중 가장 첫 번째 결과만 가져옵니다.
    p = wikipedia.page(r[0])  # 검색 결과에서 첫 번째 페이지를 가져옵니다.
    return p.content  # 해당 페이지의 내용을 반환합니다.


In [ ]:
# LLM에게 알려줄 함수 목록 정의
functions = [
    {
        "name": "search_wikipedia",  # 함수 이름을 지정합니다. 이 이름은 GPT가 함수를 호출할 때 사용됩니다.
        "description": "wikipedia 검색",  # 함수가 하는 일을 간단히 설명합니다.
        "parameters": {  # 함수가 필요로 하는 인자들을 정의합니다.
            "type": "object",  # 인자들의 데이터 형식을 객체(딕셔너리)로 설정합니다.
            "properties": {  # 각 인자의 세부 정보를 정의합니다.
                "query": {
                    "type": "string",  # 이 인자의 데이터 형식은 문자열입니다.
                    "description": "검색할 내용"  # 이 인자가 무엇을 의미하는지 설명합니다. 여기서는 검색할 내용을 입력받습니다.
                },
                "lang": {
                    "type": "string",  # 이 인자의 데이터 형식은 문자열입니다.
                    "enum": ["en", "ko"],  # 'lang' 인자는 'en'(영어) 또는 'ko'(한국어)만 허용됩니다.
                    "description": "wikipedia 언어"  # 이 인자가 무엇을 의미하는지 설명합니다. 여기서는 Wikipedia에서 사용할 언어를 지정합니다.
                }
            },
            "required": ["query"]  # 이 함수가 실행되기 위해 반드시 필요한 인자들입니다. 'query'와 'lang'이 필수적으로 입력되어야 합니다.
        }
    }
]


In [ ]:
messages = [{"role": "system", "content": "You are a helpful assistant."},
           {"role": "user", "content": "광안대교 정식 개통일이 언제야?"}]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)


In [ ]:
# 4. GPT가 요청한 함수 실행
response_message = response["choices"][0]["message"]

In [ ]:
# GPT가 함수 호출을 요청했는지 확인
if response_message.get("function_call"):
    # GPT가 호출하려는 함수의 이름을 가져옵니다.
    function_name = response_message["function_call"]["name"]

    # GPT가 함수 호출에 필요한 인자(정보)를 JSON 형식에서 파이썬 딕셔너리로 변환하여 가져옵니다.
    function_args = json.loads(response_message["function_call"]["arguments"])

    # 가져온 함수 이름과 인자를 사용하여 해당 함수를 실제로 실행합니다.
    # 여기서는 'search_wikipedia' 함수를 호출하여 Wikipedia 검색 결과를 가져옵니다.
    function_response = search_wikipedia(**function_args)

    # 5. 함수 결과를 GPT에 다시 보내기
    # GPT가 요청한 메시지를 대화 기록(messages)에 추가합니다.
    messages.append(response_message)

    # 방금 실행된 함수의 이름과 그 결과를 대화 기록(messages)에 추가합니다.
    messages.append({
        "role": "function",  # 이 메시지는 'function' 역할로 기록됩니다.
        "name": function_name,  # 실행된 함수의 이름
        "content": function_response,  # 함수 실행 결과 (예: Wikipedia에서 가져온 내용)
    })

    # 위에서 추가된 대화 기록을 기반으로 GPT에게 추가 응답을 요청합니다.
    second_response = openai.ChatCompletion.create(
        model="gpt-4o",  # GPT 모델을 지정합니다.
        messages=messages,  # 지금까지의 대화 기록을 GPT에게 보냅니다.
    )

    # 최종적으로 GPT의 응답을 출력합니다 (예: Wikipedia 검색 결과를 보여줌).
    print(second_response["choices"][0]["message"]["content"])


광안대교는 2003년 1월 6일에 정식 개통되었습니다.


https://pkgpl.org/2023/09/12/openai-api-function_call-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0/

# OpenAI 어시스턴트 API 개요



Assistants API는 AI 어시스턴트를 쉽게 구축하고 통합할 수 있게 해주는 도구입니다. 이 API는 OpenAI의 모델을 활용하여 다양한 작업을 수행하며, 코드 해석, 파일 검색, 사용자 정의 기능 호출 등의 툴을 병렬로 사용할 수 있습니다. 지속적인 대화 스레드를 통해 대화 기록을 관리하고, 파일을 생성하거나 참조할 수 있어 유연한 AI 애플리케이션 개발이 가능합니다. 현재 베타 버전으로, 지속적인 기능 추가가 이루어지고 있습니다.

사용자는 어시스턴트를 생성한 후, 대화 스레드(Thread)를 통해 특정 질문이나 요청을 보낼 수 있으며, 어시스턴트는 이에 대한 답변을 생성합니다. '런(Run)'은 어시스턴트가 스레드에서 실제로 작업을 수행하는 과정으로, 이 과정을 통해 답변이 생성되고, 최종적으로 사용자는 그 결과를 확인할 수 있습니다.

:https://platform.openai.com/playground/assistants

In [ ]:
!pip install openai --upgrade

# code_interpreter

# 🤖 GPT 어시스턴트를 활용한 수학 문제 해결 자동화: 큰 그림부터 이해하기

이 코드는 **OpenAI의 GPT-4o 모델**을 사용해 **수학 문제를 자동으로 해결해주는 어시스턴트(AI 선생님)**를 만드는 전체 과정을 담고 있습니다.  
처음 접하는 분들도 쉽게 이해할 수 있도록, **전체 흐름을 7단계로** 나눠 차근차근 설명드리겠습니다.

---

## 🧩 전체 흐름 요약: 단계별 핵심 개념

---

### ✅ 단계 1. 환경 준비
- OpenAI 라이브러리를 설치하고, API 키를 설정합니다.
- 이 API 키는 GPT와 대화를 시작하기 위한 **인증 키**입니다.

📦 비유:  
> "AI와 연결하는 전화선을 설치하고, 통화할 수 있도록 비밀번호를 입력하는 단계입니다."

---

### ✅ 단계 2. 어시스턴트 만들기
- GPT에게 **"넌 수학 문제를 푸는 선생님이야"** 라는 역할을 부여합니다.
- 파이썬 코드를 실제로 실행할 수 있는 **계산 도구(code interpreter)**를 함께 줍니다.

🎓 비유:  
> "수학 선생님을 고용하고, 계산기와 노트북을 손에 쥐어주는 것과 같습니다."

---

### ✅ 단계 3. 대화 스레드 만들기
- 사용자가 AI에게 질문을 던질 **대화 공간(스레드)**을 만듭니다.

💬 비유:  
> "질문을 종이에 적어서 선생님 책상 위에 올려놓는 과정입니다."

---

### ✅ 단계 4. 어시스턴트 실행 (Run 생성)
- 만들어진 질문(스레드)을 GPT에게 **진짜로 풀어보라고 실행시키는** 단계입니다.

🚀 비유:  
> "선생님에게 문제지를 건네고 풀기 시작하라고 지시하는 단계입니다."

---

### ✅ 단계 5. 실행 상태 확인
- AI가 문제를 **아직 푸는 중인지, 다 풀었는지, 실패했는지** 확인합니다.

🔍 비유:  
> "선생님이 아직 문제 푸는 중인지, 다 풀었는지를 확인하는 과정입니다."

---

### ✅ 단계 6. 어시스턴트의 답변 확인
- AI가 문제를 풀고 나서 **어떤 답을 냈는지, 어떻게 설명했는지**를 확인합니다.

📄 비유:  
> "책상 위에 놓인 선생님의 풀이 종이를 읽어보는 단계입니다."

---

### ✅ 단계 7. 풀이 과정 분석
- AI가 **어떤 단계와 도구(code)**를 사용해서 문제를 풀었는지 추적합니다.

🔬 비유:  
> "선생님이 문제를 풀면서 어떤 공식이나 도구를 썼는지 복기하는 과정입니다."

---



---

## 🔁 비유 정리표 (한눈에 보기)

| 단계 | 의미 | 직관적 비유 |
|------|------|--------------|
| 1 | GPT 연결 | 전화선 연결 & 비밀번호 입력 |
| 2 | 어시스턴트 생성 | 수학 선생님 고용 |
| 3 | 질문 제출 | 질문지를 책상에 올리기 |
| 4 | AI 실행 | 선생님에게 문제 넘기기 |
| 5 | 상태 확인 | 문제 푸는 중인지 보기 |
| 6 | 답변 확인 | 종이에 적힌 답 보기 |
| 7 | 풀이 분석 | 어떤 도구 썼는지 확인 |

---

## 📌 마무리 요약

이 코드는 단순히 질문을 GPT에게 던지는 것이 아닙니다.  
마치 실제 수학 선생님을 고용해서, 문제를 풀게 만들고, 그 과정을 추적하고 결과를 확인하는 전체 과정을 자동화한 것입니다.  
GPT-4o는 파이썬 코드까지 실행하며 똑똑하게 문제를 풀 수 있기 때문에, 매우 강력한 도구입니다.

이제 각 단계별로 코드를 들여다보면, 전체 구조가 명확하게 이해될 것입니다! 😊



In [ ]:
# 1. 필수 라이브러리 설치 및 API 키 설정
# os 환경 변수에 OpenAI API 키를 저장합니다.
import openai
import os

# OpenAI API 키 설정 (키 값은 실제 사용하는 키로 교체하세요)


In [ ]:
# 2. 어시스턴트 설정
# 🔸포인트: openai.beta.assistants.create 함수는 '나만의 AI 어시스턴트'를 만드는 함수입니다.
# 쉽게 말해, "이 어시스턴트는 어떤 일을 할 거야"라고 역할과 능력을 설정하는 단계입니다.
# 예: 우리가 '수학 문제를 풀어주는 AI 선생님'을 만들고자 할 때, 그 역할을 구체적으로 지정합니다.
math_assistant = openai.beta.assistants.create(
    name="MathHelper",
    # 🔸포인트: 어시스턴트의 이름입니다.
    # 예: 'MathHelper'는 수학 도우미라는 뜻으로, 이름만 봐도 무슨 일을 하는지 짐작할 수 있도록 설정합니다.
    instructions="산수 문제를 해결하는 강사입니다. 파이썬 코드를 작성하고 실행하여 답을 찾습니다.",
    # 🔸포인트: 어시스턴트의 역할과 행동 방식을 명확히 알려주는 설명입니다.
    # '산수 문제'라는 구체적인 범위와 '파이썬 코드로 답을 구한다'는 방법이 함께 정의되어 있어요.
    # 예: "나는 산수 선생님이고, 계산은 직접 파이썬으로 할 거야!"라는 식의 선언이라고 보면 됩니다.
    tools=[{"type": "code_interpreter"}],
    # 🔸포인트: 이 어시스턴트가 사용할 수 있는 도구(tool)를 지정합니다.
    # 여기서 'code_interpreter'는 파이썬 코드 실행기, 즉 코드로 직접 계산하거나 결과를 낼 수 있는 능력을 줍니다.
    # 예: 어시스턴트가 손으로 푸는 게 아니라, 컴퓨터 계산기로 문제를 푼다고 보면 됩니다.
    model="gpt-4o",
    # 🔸포인트: 어시스턴트가 기반으로 삼을 인공지능 모델입니다.
    temperature=0.2
    # 🔸포인트: 생성되는 텍스트의 "창의성" 또는 "무작위성"을 조절하는 값입니다.
    # 값이 낮을수록 더 예측 가능하고 정답에 가까운 결과를 냅니다.
    # 수학처럼 정확성이 중요한 작업에는 낮은 값을 설정하는 게 일반적입니다.
    # 예: 0.2는 거의 '정해진 방식으로 차분하게 대답하라'는 의미예요.
)


In [ ]:
math_assistant.id  # 어시스턴트의 ID 확인용 (나중에 실행에 연결할 때 사용됨)


'asst_urwSGd8RiwcrfZZLTjoQzaqx'

In [ ]:
# 3. 대화 스레드 생성
# 🔸포인트: 사용자의 질문을 담은 "대화방(스레드)"을 만드는 단계입니다.
# 이 스레드를 통해 사용자와 어시스턴트가 계속 대화하게 됩니다.

def create_thread(message):
    thread = openai.beta.threads.create(
        messages=[{"role": "user", "content": message}]  # 🔸 사용자 메시지를 포함해서 스레드 생성
    )
    return thread  # 생성된 스레드를 반환 (이후 단계에서 사용)

# 예시: 사용자의 질문으로 스레드 생성
math_thread = create_thread('100보다 큰 첫 번째 소수의 제곱은 무엇인가요?')

In [ ]:
math_thread.id  # 생성된 스레드의 ID


'thread_IIqxvSGjvjdRdvd8JWYWcqYL'

In [ ]:
# 4. 실행(run) 생성
# 🔸포인트: 어시스턴트와 스레드를 실제로 연결해서 '답변을 생성하는 실행'을 시작하는 단계입니다.

def run_assistant(thread, assistant):
    run = openai.beta.threads.runs.create(
        thread_id=thread.id,        # 연결할 스레드 ID
        assistant_id=assistant.id   # 연결할 어시스턴트 ID
    )
    return run  # 실행 객체 반환

# 어시스턴트와 스레드를 연결하여 실행 생성
math_run = run_assistant(math_thread, math_assistant)


In [ ]:
# 5. 실행 상태 확인
# 🔸포인트: 실행 중인 run이 잘 작동하고 있는지 확인하는 함수입니다.
# 실행의 상태 정보에는 진행 중, 완료됨, 오류 발생 등의 정보가 담깁니다.

def get_run_status(thread, run):
    run_status = openai.beta.threads.runs.retrieve(
        thread_id=thread.id,  # 스레드 ID
        run_id=run.id         # 실행 ID
    )
    return run_status  # 실행 상태 반환

# 실행 상태 조회
math_run_status = get_run_status(math_thread, math_run)

# 실행 상태 출력 (전체 상태와 상태값만 따로 출력)
print(math_run_status)          # 전체 실행 상태 출력
print('---')                    # 구분선
print(math_run_status.status)   # 실행 상태만 출력 (예: "completed", "in_progress")

Run(id='run_i0MsiD4NjJXacgT8tdvbimQb', assistant_id='asst_urwSGd8RiwcrfZZLTjoQzaqx', cancelled_at=None, completed_at=None, created_at=1747649350, expires_at=1747649950, failed_at=None, incomplete_details=None, instructions='산수 문제를 해결하는 강사입니다. 파이썬 코드를 작성하고 실행하여 답을 찾습니다.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1747649357, status='in_progress', thread_id='thread_IIqxvSGjvjdRdvd8JWYWcqYL', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=0.2, top_p=1.0, tool_resources={}, reasoning_effort=None)
---
in_progress


In [ ]:
# 6. 스레드 메시지 확인
# 🔸포인트: 어시스턴트가 생성한 답변 메시지를 확인하는 함수입니다.
# 이 과정을 통해 실제로 어떤 응답이 생성되었는지 사용자 입장에서 볼 수 있습니다.

def list_thread_messages(thread):
    messages = openai.beta.threads.messages.list(
        thread_id=thread.id  # 해당 스레드의 ID로 메시지 가져오기
    )

    # 최신 메시지부터 순서대로 출력
    for i in range(len(messages.data), 0, -1):
        print(messages.data[i-1].role)  # 메시지 주체 (user 또는 assistant)
        print(messages.data[i-1].content[0].text.value)  # 메시지 내용 출력
        print('---')  # 구분선 출력

    return messages  # 전체 메시지 반환

# 메시지 목록 조회 및 출력
math_messages = list_thread_messages(math_thread)


user
100보다 큰 첫 번째 소수의 제곱은 무엇인가요?
---


In [ ]:
# 7. 실행 단계 확인
# 🔸포인트: 어시스턴트가 답변을 생성하는 과정에서 어떤 단계를 거쳤는지 확인합니다.
# 디버깅이나 동작 확인 시 매우 유용합니다. (예: 코드 실행, 텍스트 생성 등)

run_steps = openai.beta.threads.runs.steps.list(
  thread_id=math_thread.id,        # 스레드 ID
  run_id=math_run_status.id        # 실행(run) ID
)

# 각 단계에서 어떤 작업이 수행되었는지 출력
for i in range(len(run_steps.data), 0, -1):
    step_detail = run_steps.data[i-1].step_details  # 단계 세부 정보 가져오기
    print(step_detail)  # 단계 정보 출력
    print('---')  # 구분선

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_B4MkN7XKOTwAg9rc0w5zeU2r'), type='message_creation')
---
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_i10L7iCWAgSeROFCXPvqEovD', code_interpreter=CodeInterpreter(input='import sympy\n\n# 100보다 큰 첫 번째 소수를 찾습니다.\nfirst_prime_over_100 = sympy.nextprime(100)\n\n# 그 소수의 제곱을 계산합니다.\nsquare_of_first_prime_over_100 = first_prime_over_100 ** 2\nfirst_prime_over_100, square_of_first_prime_over_100', outputs=[]), type='code_interpreter')], type='tool_calls')
---
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_a8jsx3lTUmiPsugigKawjT0u'), type='message_creation')
---


# 어시스턴트에 파일 추가 및 간단한 QA 챗봇 만들기(수정하기)





 OpenAI의 클라이언트를 사용하여 파일을 업로드하고, 해당 파일을 기반으로 간단한 QA 챗봇을 만드는 과정을 구현합니다. 해당 파일은 어시스턴트의 지식 베이스로 활용되며, 특정 질문에 답변하는 데 사용됩니다.


In [ ]:
# 벡터 스토어 생성
vector_store = openai.beta.vector_stores.create(name='축구')
# 🔸포인트: 벡터 스토어는 문서를 "미리 학습된 검색용 데이터베이스"처럼 저장하는 공간입니다.
# 예: '축구'라는 이름의 문서 보관함을 만든다고 이해하면 됩니다.
# 나중에 챗봇이 이 안에서 관련 정보를 찾아 답변하게 됩니다.

file_path = '/content/rule.txt'  # 업로드할 텍스트 파일 경로 설정
file_streams = [open(file_path, 'rb')]  # 🔸 바이너리 모드로 파일을 읽어서 리스트로 준비

# 파일 업로드 및 벡터 스토어에 추가
file_batch = openai.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id,  # 위에서 만든 벡터 스토어의 ID 지정
  files=file_streams  # 업로드할 파일 리스트 전달
)
# 🔸포인트: 파일을 업로드하면, 그 내용을 AI가 인덱싱(검색용으로 분석)해서 저장합니다.
# 나중에 '검색 기반 답변'이 가능해지는 핵심 단계입니다.

In [ ]:
# 간단한 QA 챗봇 생성
soccer_assistant = openai.beta.assistants.create(
  name='축구 도우미',  # 챗봇 이름: '축구 도우미'
  instructions="당신은 축구 전문 봇입니다. 파일로 주어진 지식 베이스를 이용하여 질문에 답변하세요.",
  # 🔸포인트: 챗봇의 역할을 설명하는 부분
  # 예: “나는 축구 전문가야, 그리고 문서 내용 기반으로 대답할게” 라고 설정하는 문장입니다.
  model="gpt-3.5-turbo-0125",  # 사용할 AI 모델 설정 (GPT-3.5 버전)
  tools=[{"type": "file_search"}]  # 🔸포인트: 파일 내용을 검색할 수 있는 도구 활성화
)

soccer_assistant  # 생성된 챗봇 객체 (정보 확인용)

In [ ]:
# 챗봇에 벡터 스토어 연결
soccer_assistant = openai.beta.assistants.update(
  assistant_id=soccer_assistant.id,  # 어떤 챗봇을 업데이트할지 ID 지정
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
  # 🔸포인트: 챗봇이 어떤 파일 검색 공간을 사용할지를 연결하는 부분
  # 예: 축구 관련 정보를 어디서 찾을지를 알려주는 설정입니다.
)

In [ ]:

# 스레드(대화 주제) 생성
soccer_thread = create_thread("축구는 몇명이서 하지?")
# 🔸포인트: 사용자 질문을 담은 "대화방"을 생성하는 단계
# 이 스레드를 통해 사용자의 질문과 챗봇의 답변이 이어집니다.

soccer_thread


In [ ]:
soccer_assistant

In [ ]:
# 스레드 실행을 위한 함수 정의
def create_run(thread, assistant):
    # 지정된 스레드와 챗봇을 사용하여 대화 실행 생성
    run = openai.beta.threads.runs.create(
        thread_id=thread.id,  # 실행할 대화 스레드 ID
        assistant_id=assistant.id  # 사용할 챗봇 ID
    )
    return run  # 실행 결과 반환


In [ ]:

# 스레드 실행: 사용자의 질문에 대해 챗봇이 실제 응답하도록 처리
soccer_run = create_run(soccer_thread, soccer_assistant)
soccer_run  # 실행 정보 확인

In [ ]:
# 메시지 목록을 출력하는 함수 정의
def list_threads_messages(thread, print_content=True):
    # 지정된 스레드의 모든 메시지 가져오기
    messages = openai.beta.threads.messages.list(
        thread_id=thread.id  # 어떤 스레드의 메시지를 조회할지 ID 지정
    )
    if print_content:  # 출력 여부 확인
        for i in range(len(messages.data), 0, -1):  # 메시지들을 최신순으로 출력
            print(messages.data[i-1].role)  # 메시지 주체 ('user' 또는 'assistant')
            print(messages.data[i-1].content[0].text.value)  # 실제 메시지 내용 출력
            print('---')  # 구분선 출력 (시각적으로 보기 좋게)

    return messages  # 전체 메시지 반환


In [ ]:

# 챗봇이 사용자 질문에 대해 어떤 답을 했는지 확인
soccer_messages = list_threads_messages(soccer_thread)

## OPENAI Assistant Functional Calling

| 포인트                  | 설명                                    |
| -------------------- | ------------------------------------- |
| `assistant.create()` | 챗봇의 역할과 도구를 설정하는 핵심 함수                |
| `thread`             | 사용자와 챗봇의 대화 흐름을 담는 구조                 |
| `function tool`      | 챗봇이 특정 작업을 외부 API로 위임하는 메커니즘          |
| `event_handler`      | 도우미가 실행 중 함수 실행을 요청할 때 이를 감지하고 처리     |
| `stream()`           | 실시간 응답 처리를 위한 방식 (사용자에게 자연스러운 피드백 제공) |


In [ ]:
# 📌 필수 라이브러리 설치 및 환경 설정

# OpenAI API와 통신하기 위한 핵심 라이브러리 임포트
from openai import OpenAI  # 🔸포인트: OpenAI API 클라이언트를 사용하기 위한 클래스
from openai import AssistantEventHandler  # 이벤트를 처리할 때 사용하는 핸들러 클래스
from typing_extensions import override  # 🔸포인트: 부모 클래스 메서드를 덮어쓸 때 사용
import openai  # OpenAI의 다양한 기능을 활용할 수 있는 메인 라이브러리
import os  # OS 환경변수나 파일 경로 설정 등을 위해 사용

# OpenAI API 키 설정 (개인 키를 안전하게 보관해야 함)
# os.environ['OPENAI_API_KEY'] = "sk-..."  # 실제 강의 시엔 키를 노출하지 마세요!

# OpenAI 클라이언트 객체 생성
client = openai.OpenAI()  # 이후 모든 API 호출은 이 client를 통해 이루어짐

# 📌 Assistant(도우미) 정의

assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    # 🔸포인트: 도우미의 역할을 명시적으로 정의
    # 🧠 "나는 날씨를 알려주는 봇이야. 그리고 이 아래 정의된 함수들을 이용해서 대답할 거야."

    model="gpt-4o",  # 최신 GPT-4o 모델 사용

    tools=[  # 도우미가 사용할 수 있는 함수(도구) 정의
        {
            "type": "function",
            "function": {
                "name": "get_current_temperature",  # 온도 조회 함수
                "description": "Get the current temperature for a specific location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g., San Francisco, CA"
                        },
                        "unit": {
                            "type": "string",
                            "enum": ["Celsius", "Fahrenheit"],  # 🔸섭씨 또는 화씨 중 선택
                            "description": "The temperature unit to use."
                        }
                    },
                    "required": ["location", "unit"],  # 필수 입력값
                    "additionalProperties": False  # 그 외 속성은 허용 안 함
                },
                "strict": True
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_rain_probability",  # 비 확률 조회 함수
                "description": "Get the probability of rain for a specific location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g., San Francisco, CA"
                        }
                    },
                    "required": ["location"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    ]
)

In [ ]:
# 📌 사용자와 대화를 위한 스레드 생성

thread = client.beta.threads.create()  # 🔸포인트: 대화는 항상 스레드 안에서 이루어짐

# 사용자 질문 추가
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # 대화 스레드 ID 지정
    role="user",  # 이 메시지가 사용자로부터 왔음을 명시
    content="What's the weather in San Francisco today and the likelihood it'll rain?"
    # 💡예시: 오늘 샌프란시스코 날씨와 비 올 확률을 물어봄
)

(참고) 사용자가 특정 질문을 입력하고, 이 질문에 대한 처리가 필요한 경우, 시스템은 "어떤 작업이 필요하다"는 이벤트를 발생시킵니다.
이때 이벤트 시스템은 'thread.run.requires_action' 이벤트를 발생시킬 수 있으며, 이 이벤트가 발생하면 시스템은 EventHandler 클래스의 on_event 메서드를 호

In [ ]:
# 📌 이벤트 핸들러 정의: 도우미가 기능 실행 요청할 때 처리해주는 클래스

class EventHandler(AssistantEventHandler):
    # 🔸포인트: AssistantEventHandler는 OpenAI 도우미와 상호작용할 때 발생하는 이벤트를 감지하고 처리하는 '이벤트 리스너'입니다.
    # 이 클래스를 상속받아 우리가 원하는 방식으로 이벤트를 처리할 수 있게 됩니다.

    @override
    def on_event(self, event):
        # 🔸포인트: 이벤트 발생 시 자동으로 호출되는 메서드입니다.
        # 도우미가 "나 함수 좀 실행해야 해!"라고 요청하는 상황을 감지합니다.
        # 예: 사용자가 날씨 질문 → 도우미가 함수 호출이 필요하다고 판단 → 이 메서드가 실행됨

        if event.event == 'thread.run.requires_action':
            # 💡특정 이벤트 타입: '추가 작업 필요' (즉, 함수 실행 요청)
            run_id = event.data.id  # 현재 실행(run)의 ID를 가져옴
            self.handle_requires_action(event.data, run_id)  # 작업 처리 메서드 호출

    def handle_requires_action(self, data, run_id):
        # 🔸포인트: 도우미가 요청한 실제 함수 실행을 담당하는 메서드
        # 이 안에서 도우미가 호출한 함수 이름에 따라 우리가 직접 결과를 만들어 넘깁니다.

        tool_outputs = []  # 도구의 출력값(함수 결과들)을 담을 리스트

        # 요청된 함수(도구)들을 하나씩 순회하며 확인
        for tool in data.required_action.submit_tool_outputs.tool_calls:

            if tool.function.name == "get_current_temperature":
                # 💡도우미가 "현재 온도를 알려줘!"라고 요청한 경우
                tool_outputs.append({
                    "tool_call_id": tool.id,  # 요청된 함수 고유 ID
                    "output": "57"  # 예시로 57도라는 결과를 직접 지정
                    # 실제로는 날씨 API를 호출하여 받아온 값을 넣을 수 있음
                })

            elif tool.function.name == "get_rain_probability":
                # 💡도우미가 "비 올 확률이 얼마나 돼?"라고 요청한 경우
                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": "0.06"  # 예시로 6%라는 확률을 리턴
                })

        # 🔸포인트: 도우미가 요청한 함수들에 대한 결과들을 정리한 뒤, 아래 메서드를 통해 서버에 제출
        self.submit_tool_outputs(tool_outputs, run_id)

    def submit_tool_outputs(self, tool_outputs, run_id):
        # 🔸포인트: 앞서 만든 함수 결과들을 도우미에게 제출하는 단계입니다.
        # 제출 방식은 스트리밍을 사용하여 결과를 실시간으로 보여줄 수 있게 처리합니다.

        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=thread.id,      # 현재 대화의 스레드 ID
            run_id=run_id,            # 현재 실행의 ID
            tool_outputs=tool_outputs,  # 도구 출력값 리스트
            event_handler=EventHandler(),  # 이 핸들러를 다시 지정해줌 (다음 이벤트도 처리 가능)
        ) as stream:
            # 도우미가 결과를 사용자에게 전달할 때, 그 내용을 실시간으로 받아 출력
            for text in stream.text_deltas:
                print(text, end="", flush=True)  # 실시간 출력
            print()  # 보기 좋게 줄바꿈


In [ ]:

# 📌 Run 실행 및 결과 스트리밍

with client.beta.threads.runs.stream(
    thread_id=thread.id,  # 실행할 스레드 지정
    assistant_id=assistant.id,  # 사용할 도우미 ID 지정
    event_handler=EventHandler()  # 핸들러 지정
) as stream:
    stream.until_done()  # 모든 응답이 도착할 때까지 대기